<a href="https://colab.research.google.com/github/sanmevv/Turkey-Syria-Earthquake-Disaster-management-/blob/main/Turkey_earthquake_SimpleRNN%2CLSTM_%26_GRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall keras

Found existing installation: keras 2.11.0
Uninstalling keras-2.11.0:
  Would remove:
    /usr/local/lib/python3.9/dist-packages/keras-2.11.0.dist-info/*
    /usr/local/lib/python3.9/dist-packages/keras/*
Proceed (Y/n)? ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/pip/_internal/cli/base_command.py", line 167, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.9/dist-packages/pip/_internal/commands/uninstall.py", line 97, in run
    uninstall_pathset = req.uninstall(
  File "/usr/local/lib/python3.9/dist-packages/pip/_internal/req/req_install.py", line 638, in uninstall
    uninstalled_pathset.remove(auto_confirm, verbose)
  File "/usr/local/lib/python3.9/dist-packages/pip/_internal/req/req_uninstall.py", line 363, in remove
    if auto_confirm or self._allowed_to_proceed(verbose):
  File "/usr/local/lib/python3.9/dist-packages/pip/_internal/req/req_uninstall.py", line 403, in _allowed

In [2]:
#!pip uninstall keras
!pip install keras


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install --upgrade keras


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip uninstall keras
!pip install tensorflow
from tensorflow.keras.preprocessing.sequence import pad_sequences




In [5]:
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [ ]:
import tensorflow.keras as keras

In [ ]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.pyplot import xticks
from nltk.corpus import stopwords
import nltk
import re
from nltk.stem import WordNetLemmatizer
import string
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
from collections import defaultdict
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping, Callback
import tensorflow as tf
from sklearn.metrics import f1_score
from wordcloud import WordCloud,STOPWORDS
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
!pip install keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense,Flatten,Embedding,Activation,Dropout
from keras.layers import Conv1D,MaxPooling1D,GlobalMaxPooling1D,LSTM
from keras.layers import Bidirectional

In [ ]:
# load train and test datasets
train= pd.read_csv('/content/drive/MyDrive/train_data.csv')
test=pd.read_csv('/content/drive/MyDrive/test_data.csv')

In [ ]:
# check the no. of rows and columns in the dataset
train.shape, test.shape

In [ ]:
train.head()

In [ ]:
train


In [ ]:
test.drop('Index', axis=1, inplace=True)


In [ ]:
test

In [ ]:
train.isnull().sum().sort_values(ascending = False)

In [ ]:
# Define the keywords that indicate a tweet is disaster-related

disaster_keywords = ['earthquake', 'Turkey', 'Syria', 'magnitude', 'tremor', 'seismic', 'Richter scale', 'epicenter', 'aftershock', 'disaster', 'rescue', 'relief', 'casualties', 'injured', 'damages', 'buildings', 'infrastructure', 'natural disaster', 'emergency', 'humanitarian']
# Create a new column called "Disaster-related" and set its value to True if any of the disaster keywords appear in the tweet, otherwise set it to False
train['Target'] = train['Tweet'].apply(lambda x: any(keyword in x.lower() for keyword in disaster_keywords))

# View the updated dataframe
train.head()

In [ ]:
sns.countplot(x=train.Target)

Data Cleaning

In [ ]:
#lets save stopwords in a variable
import nltk
nltk.download('stopwords')
stop = list(stopwords.words("english"))

In [ ]:
# save list of punctuation/special characters in a variable
punctuation = list(string.punctuation)


In [ ]:
# create an object to convert the words to its lemma form
lemma = WordNetLemmatizer()


In [ ]:
# lets make a combine list of stopwords and punctuations
sw_pun = stop + punctuation

In [ ]:
# function to preprocess the messages
def preprocess(tweet):
    tweet = re.sub(r"https?:\/\/t.co\/[A-Za-z0-9]+", "", tweet) # removing urls 
    tweet = re.sub('[^\w]',' ',tweet) # remove embedded special characters in words (for example #earthquake)         
    tweet = re.sub('[\d]','',tweet) # this will remove numeric characters
    tweet = tweet.lower()
    words = tweet.split()  
    sentence = ""
    for word in words:     
        if word not in (sw_pun):  # removing stopwords & punctuations                
            word = lemma.lemmatize(word,pos = 'v')  # converting to lemma    
            if len(word) > 3: # we will consider words with length  greater than 3 only
                sentence = sentence + word + ' '             
    return(sentence)

In [ ]:
# apply preprocessing functions on the train and test datasets
import nltk
nltk.download('wordnet')
train['Tweet'] = train['Tweet'].apply(lambda s : preprocess(s))
test ['Tweet'] = test ['Tweet'].apply(lambda s : preprocess(s))

In [ ]:
# function to remove emojis
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

In [ ]:
# applying the function on the train and the test datasets
train['Tweet'] = train['Tweet'].apply(lambda s : remove_emoji(s))
test ['Tweet'] = test ['Tweet'].apply(lambda s : remove_emoji(s))

Vocabulary creation

In [ ]:
# function to create vocab
from collections import Counter
def create_vocab(train):
    vocab = Counter()
    for i in range(train.shape[0]):
        vocab.update(train.Tweet[i].split())
    return(vocab)

In [ ]:
# concatenate training and testing datasets
master=pd.concat((train,test)).reset_index(drop=True)

# call vocabulary creation function on master dataset
vocab = create_vocab(master)

# lets check the no. of words in the vocabulary
len(vocab)

In [ ]:
# lets check the most common 50 words in the vocabulary
vocab.most_common(50)

In [ ]:
# create the final vocab by considering words with more than one occurence
final_vocab = []
min_occur = 2
for k,v in vocab.items():
    if v >= min_occur:
        final_vocab.append(k)

In [ ]:
# lets check the no. of the words in the final vocabulary
vocab_size = len(final_vocab)
vocab_size

In [ ]:
# function to filter the dataset, keep only words which are present in the vocab
def filter(tweet):
    sentence = ""
    for word in tweet.split():  
        if word in final_vocab:
            sentence = sentence + word + ' '
    return(sentence)

In [ ]:
# apply filter function on the train and test datasets
train['Tweet'] = train['Tweet'].apply(lambda s : filter(s))
test ['Tweet'] = test ['Tweet'].apply(lambda s : filter(s))

In [ ]:
# lets take a look at the update training dataset
train.Tweet.head()

Data Preprocessing

In [ ]:
# the different units into which you can break down text (words, characters, or n-grams) are called tokens, 
# and breaking text into such tokens is called tokenization, this can be achieved using Tokenizer in Keras

from keras.preprocessing.text import Tokenizer
# fit a tokenizer
def create_tokenizer(lines):
    # num_words = vocab_size will create a tokenizer,configured to only take into account the vocab_size(6025)
    tokenizer = Tokenizer(num_words=vocab_size)
    # Build th word index, Turns strings into lists of integer indices
    tokenizer.fit_on_texts(lines) 
    return tokenizer

In [ ]:
# create and apply tokenizer on the training dataset
tokenizer = create_tokenizer(train.Tweet)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

In [ ]:
# converting texts into vectors
train_text = tokenizer.texts_to_matrix(train.Tweet, mode = 'freq')

Model Building & Evaluation

1. Neural Network

In [ ]:
# function to calculate f1 score for each epoch
import keras.backend as K
def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [ ]:
# define the model
def define_model(n_words):
    # define network
    model = Sequential()
    model.add(Dense(1024, input_shape=(n_words,), activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(512,activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(256,activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(1, activation='sigmoid'))
    
    # compile network
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics = [get_f1])
    
    # summarize defined model
    model.summary()
    return model

In [ ]:
X_train.shape

In [ ]:
callbacks_list = [EarlyStopping(monitor='get_f1',patience=10,),
ModelCheckpoint(filepath='./NN.h5',monitor='val_loss',save_best_only=True)
]

In [ ]:
# create the model
n_words = X_train.shape[1]
model = define_model(n_words)

In [ ]:
#fit network
history = model.fit(X_train,y_train,epochs=100,verbose=2,callbacks=callbacks_list,validation_split=0.2)

In [ ]:
acc = history.history['get_f1']
val_acc = history.history['val_get_f1']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
import keras

dependencies = {
    'get_f1': get_f1
}


# load the model from disk
loaded_model_NN = keras.models.load_model('./NN.h5',custom_objects=dependencies)

In [ ]:
# prediction on the test dataset
#X_test_Set = tokenizer.texts_to_matrix(X_test, mode = 'freq')
y_pred = loaded_model_NN.predict_classes(X_test)

In [ ]:
# important metrices
print(classification_report(y_test, y_pred))

Predictions on the test dataset

In [ ]:
test_id = test.id
test.drop(["id","location","keyword"],1,inplace = True)

# apply tokenizer on the test dataset
test_set = tokenizer.texts_to_matrix(test.text, mode = 'freq')

In [ ]:
# make predictions on the test dataset
y_test_pred = loaded_model_NN.predict_classes(test_set)

In [ ]:
# lets prepare for the prediction submission
sub = pd.DataFrame()
sub['Id'] = test_id
sub['target'] = y_test_pred
sub.head()

In [ ]:
sub.to_csv('submission_NN.csv',index=False)